In [2]:
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
def denoise_nav_depth(df: pd.DataFrame, thresh=6, iters=1) -> pd.DataFrame:
    """
    Remove outlier noisy depth values that are adjacent to valid values iteratively.

    The assumption of the noise is that the depth noise only jumps to shallower depths
    erroneously.
    """
    valid_nav = df
    for _ in range(iters):
        good = np.diff(valid_nav["depth"]) > -thresh
        good = np.insert(good, 0, True)
        good2 = np.diff(valid_nav["depth"]) < thresh
        good2 = np.insert(good2, -1, True)
        valid_nav = valid_nav[good & good2]
    return valid_nav

def apply_clahe(img, clipLimit=None, tileGridSize=None):
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=clipLimit, tileGridSize=tileGridSize)
    cl = clahe.apply(l)
    limg = cv2.merge((cl, a, b))
    final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
    return final

In [12]:
# vidpath = Path("../data/dive-data/Dive8/clips/barrel1.mp4")
# start_t = pd.Timestamp("2023-04-14T11:15:24", tz="US/Pacific")
# vidpath = Path("../data/dive-data/Dive8/clips/barrelddt1.mp4")
# start_t = pd.Timestamp("2023-04-14T14:26:54", tz="US/Pacific")
# vidpath = Path("../dive-data/Dive8/clips/barreltriplet.mp4")
# start_t = pd.Timestamp("2023-04-14T14:55:46", tz="US/Pacific")
# vidpath = Path("../data/dive-data/Dive8/clips/barrel2.mp4")
# start_t = pd.Timestamp("2023-04-14T16:15:58", tz="US/Pacific")
# vidpath = Path("../dive-data/Dive8/clips/barrelddt2.mp4")
# start_t = pd.Timestamp("2023-04-14T14:43:22", tz="US/Pacific")
# vidpath = Path("../dive-data/Dive8/clips/barrel3.mp4")
# start_t = pd.Timestamp("2023-04-14T15:02:42", tz="US/Pacific")
# vidpath = Path("../data/dive-data/Dive8/clips/barrelddt3.mp4")
# start_t = pd.Timestamp("2023-04-14T15:05:23", tz="US/Pacific")
# vidpath = Path("../data/dive-data/Dive8/clips/barrelddt4.mp4")
# start_t = pd.Timestamp("2023-04-14T13:07:36", tz="US/Pacific")
vidpath = Path("../data/dive-data/Dive8/clips/barrel4.mp4")
start_t = pd.Timestamp("2023-04-14T13:04:49", tz="US/Pacific")
# vidpath = Path("../data/dive-data/Dive8/clips/seafloor.mp4")
# start_t = pd.Timestamp("2023-04-14T12:24:20", tz="US/Pacific")
# vidpath = Path("../data/dive-data/Dive8/clips/dive8-barrel-15-04.mp4")
# start_t = pd.Timestamp("2023-04-14T15:05:46", tz="US/Pacific")
# vidpath = Path("../data/dive-data/Dive8/clips/dive8-barrel-15-14.mp4")
# start_t = pd.Timestamp("2023-04-14T15:15:39", tz="US/Pacific")
# vidpath = Path("../data/dive-data/Dive8/clips/dive8-barrel-10-45.mp4")
# start_t = pd.Timestamp("2023-04-14T10:46:31", tz="US/Pacific")
# vidpath = Path("../data/dive-data/Dive8/clips/dive8-barrel-10-49.mp4")
# start_t = pd.Timestamp("2023-04-14T10:49:31", tz="US/Pacific")
# vidpath = Path("../data/dive-data/Dive8/clips/dive8-barrel-11-04.mp4")
# start_t = pd.Timestamp("2023-04-14T11:04:09", tz="US/Pacific")

if not vidpath.exists():
    raise FileNotFoundError(f"Video file {vidpath} not found")
navpath = Path("../data/dive-data/Dive8/ROV Dive 8 nav.csv")
outdir = vidpath.parent / vidpath.stem

crop = True
# top left x, y, width, height
bbox = [0, 120, 1920, 875]
# nav depth data is extremely buggy, values just jump up for no reason
depth_noise_thresh = 6

outdir.mkdir(parents=True, exist_ok=True)
vid = cv2.VideoCapture(str(vidpath))
# videos are at 25 fps
fps = 25
step = 50
dt = pd.Timedelta(seconds=step / fps)
nav = pd.read_csv(navpath)
nav["timestamp"] = pd.to_datetime(nav["timestamp"])
valid_nav = denoise_nav_depth(nav, thresh=depth_noise_thresh, iters=2)
curr_t = start_t
nframes = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
increase_contrast = True

fnames = []
times = []
nav_times = []
lats = []
lons = []
depths = []  # meters
yaws = []  # radians, converted from degree heading measurement relative to north
pitches = []  # radians, converted from being relative to vehicle
rolls = []  # radians
cnt = 0
for i in range(0, nframes, step):
    vid.set(cv2.CAP_PROP_POS_FRAMES, i)
    _, frame = vid.read()
    if frame is None:
        # video ended before expected number of frames
        break
    if increase_contrast:
        frame = apply_clahe(frame, clipLimit=2.0, tileGridSize=(8, 8))
    if crop:
        cropped = frame[bbox[1]:bbox[1]+bbox[3], bbox[0]:bbox[0]+bbox[2]]
        fname = f"cropped{str(cnt).zfill(4)}.jpg"
    else:
        cropped = frame
        fname = f"uncropped{str(cnt).zfill(4)}.jpg"
    navrow = valid_nav[valid_nav["timestamp"] <= curr_t].iloc[-1]
    fnames.append(fname)
    times.append(curr_t)
    nav_times.append(navrow["timestamp"])
    lats.append(navrow["latitude"])
    lons.append(navrow["longitude"])
    depths.append(navrow["depth"])
    # convert from degrees to radians
    # change yaw/pitch to be relative to camera, not vehicle
    # yaw is not bearing anymore, so counterclockwise instead
    yaws.append((90 - navrow["heading"]) * np.pi / 180)
    pitches.append((navrow["pitch"] + 90) * np.pi / 180)
    rolls.append(navrow["roll"] * np.pi / 180)
    curr_t += dt
    cv2.imwrite(str(outdir / fname), cropped)
    cnt += 1
df = pd.DataFrame({
    "filename": fnames,
    "timestamp": times,
    "nav_timestamp": nav_times,
    "latitude": lats,
    "longitude": lons,
    "depth": depths,
    "yaw": yaws,
    "pitch": pitches,
    "roll": rolls
})
df.to_csv(outdir / "frame-time-nav.csv", index=False)